In [1]:
import csv
from itertools import islice, chain
from collections import namedtuple
from datetime import datetime
from operator import attrgetter
from functools import reduce
from operator import add

In [2]:
# Defining the datatypes in the row
data_types_vehicles = ['STRING', 'STRING', 'STRING', 'DATE']
data_types_upstatus = ['STRING', 'DATETIME', 'DATETIME']
data_types_perinfo = ['STRING', 'STRING', 'STRING', 'STRING','STRING']
data_types_employ = ['STRING', 'STRING', 'STRING', 'STRING']

In [3]:
# Function to cast the value to the appropriate datatype
def cast(data_type, value):
    if data_type == 'DOUBLE':
        return float(value)
    elif data_type == 'INT':
        return int(value)
    elif data_type =='DATETIME':
        return datetime.strptime(value,'%Y-%m-%dT%H:%M:%S%z').date()
    elif data_type == 'DATE':
        return datetime.strptime(value,'%Y').date().year
    else:
        return str(value)

In [4]:
#Function which takes one row and sends the elements one by one for casting. Returns a list of casted elements 
def cast_row(data_types, data_row):
    return [cast(data_type, value) 
            for data_type, value in zip(data_types, data_row)]

In [5]:
# reading the file provided by filename and returning an iterator
def read_file(filename, data_types, header = False):
    with open(filename) as file:
        file_iter = iter(file)
        headers = next(file_iter).strip('\n').split(',')
        tuplename = filename.split('.')[0]
        named_tuple = namedtuple(tuplename, headers, defaults=(None,) * len(headers))
        if header:
            yield headers
        for line in file_iter:
            data = line.strip('\n').split(',')
            data = cast_row(data_types, data)
            iterdata = named_tuple(*data)
            yield iterdata

## Goal1: Your first task is to create iterators for each of the four files that contained cleaned up data, of the correct type (e.g. string, int, date, etc), and represented by a named tuple.

In [6]:
# Creating an iterator for personal_info.csv using read_file() and printing the first 5 elements 
rows_perinfo = read_file('personal_info.csv', data_types_perinfo)
for row in islice(rows_perinfo,6):
    print(row)

personal_info(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
personal_info(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
personal_info(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')
personal_info(ssn='104-22-0928', first_name='Justinian', last_name='Kunzelmann', gender='Male', language='Dhivehi')
personal_info(ssn='104-84-7144', first_name='Claudianus', last_name='Brixey', gender='Male', language='Afrikaans')
personal_info(ssn='105-27-5541', first_name='Federico', last_name='Aggett', gender='Male', language='Chinese')


In [7]:
# Creating an iterator for employment.csv using read_file() and printing the first 5 elements
rows_employ = read_file('employment.csv', data_types_employ)
for row in islice(rows_employ, 6):
    print(row)

employment(employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', ssn='100-53-9824')
employment(employer='Nicolas and Sons', department='Sales', employee_id='41-6841359', ssn='101-71-4702')
employment(employer='Connelly Group', department='Research and Development', employee_id='98-7952860', ssn='101-84-0356')
employment(employer='Upton LLC', department='Marketing', employee_id='56-9817552', ssn='104-22-0928')
employment(employer='Zemlak-Olson', department='Business Development', employee_id='46-2886707', ssn='104-84-7144')
employment(employer='Kohler  Bradtke and Davis', department='Support', employee_id='80-0975518', ssn='105-27-5541')


In [8]:
# Creating an iterator for update_status.csv using read_file() and printing the first 5 elements
rows_updateStatus = read_file('update_status.csv', data_types_upstatus)
for row in islice(rows_updateStatus, 6):
    print(row)

update_status(ssn='100-53-9824', last_updated=datetime.date(2017, 10, 7), created=datetime.date(2016, 1, 24))
update_status(ssn='101-71-4702', last_updated=datetime.date(2017, 1, 23), created=datetime.date(2016, 1, 27))
update_status(ssn='101-84-0356', last_updated=datetime.date(2017, 10, 4), created=datetime.date(2016, 9, 21))
update_status(ssn='104-22-0928', last_updated=datetime.date(2017, 3, 28), created=datetime.date(2016, 4, 15))
update_status(ssn='104-84-7144', last_updated=datetime.date(2018, 2, 19), created=datetime.date(2016, 3, 15))
update_status(ssn='105-27-5541', last_updated=datetime.date(2017, 7, 24), created=datetime.date(2016, 7, 23))


In [9]:
# Creating an iterator for vehicles.csv using read_file() and printing the first 5 elements
rows_vehicles = read_file('vehicles.csv', data_types_vehicles)
for row in islice(rows_vehicles, 6):
    print(row)

vehicles(ssn='100-53-9824', vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)
vehicles(ssn='101-71-4702', vehicle_make='Ford', vehicle_model='Mustang', model_year=1997)
vehicles(ssn='101-84-0356', vehicle_make='GMC', vehicle_model='Yukon', model_year=2005)
vehicles(ssn='104-22-0928', vehicle_make='Oldsmobile', vehicle_model='Intrigue', model_year=2000)
vehicles(ssn='104-84-7144', vehicle_make='Ford', vehicle_model='Crown Victoria', model_year=2008)
vehicles(ssn='105-27-5541', vehicle_make='Ford', vehicle_model='Mustang', model_year=2001)


## Goal 2:
Create a single iterable that combines all the columns from all the iterators.

The iterable should yield named tuples containing all the columns.
Make sure that the SSN's across the files match!

All the files are guaranteed to be in SSN sort order, and every SSN is unique, and every SSN appears in every file.

Make sure the SSN is not repeated 4 times - one time per row is enough!

In [10]:
## Function for sorting the iterator and returns a sorted iterator
def sorting(iterator, field):
    sorted_ = iter(sorted(iterator, key=lambda x: getattr(x, field), reverse = False))
    return (sorted_)

In [11]:
## Function for merging the individual namedtuple in a list
def merge(ntuples):
    m = {}
    for i in ntuples:
        m.update(i._asdict())
    M = namedtuple("Combiter", m.keys())(*m.values())
    return M

In [12]:
## Combining the iterators and returning the value via a yield thus making the function a generator
def combiter(*args):
    for k in zip(*args):
        yield merge(list(k))

In [13]:
## Defining Individual Iterators
rows_perinfo = read_file('personal_info.csv', data_types_perinfo, header= True)
rows_employ = read_file('employment.csv', data_types_employ, header = True)
rows_updateStatus = read_file('update_status.csv', data_types_upstatus, header = True)
rows_vehicles = read_file('vehicles.csv', data_types_vehicles, header= True)

In [14]:
## Obtaining the headers and printing the headers
h_perinfo = next(rows_perinfo)
h_employ  = next(rows_employ)
h_updateStatus = next(rows_updateStatus)
h_vehicles = next(rows_vehicles)
print(h_perinfo,h_employ,h_updateStatus,h_vehicles)

['ssn', 'first_name', 'last_name', 'gender', 'language'] ['employer', 'department', 'employee_id', 'ssn'] ['ssn', 'last_updated', 'created'] ['ssn', 'vehicle_make', 'vehicle_model', 'model_year']


In [15]:
## Sorting the iterators w.r.t ssn
rows_perinfo_sort = sorting(rows_perinfo, 'ssn')
rows_employ_sort = sorting(rows_employ,'ssn')
rows_updateStatus_sort = sorting(rows_updateStatus,'ssn')
rows_vehicles_sort = sorting(rows_vehicles,'ssn')

In [16]:
#for rows in islice(rows_perinfo_sort,5):
    #print(rows)

In [17]:
#for rows in islice(rows_employ_sort,5):
    #print(rows)

In [18]:
#for rows in islice(rows_updateStatus_sort,5):
    #print(rows)

In [19]:
#for rows in islice(rows_vehicles_sort,5):
    #print(rows)

In [20]:
# Printing the first 5 elements from the combined iterators
for index, rows in enumerate(islice(combiter(rows_perinfo_sort,rows_employ_sort, rows_updateStatus_sort, rows_vehicles_sort), 5)):
    print(index,'-->',rows)

0 --> Combiter(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic', employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', last_updated=datetime.date(2017, 10, 7), created=datetime.date(2016, 1, 24), vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)
1 --> Combiter(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao', employer='Nicolas and Sons', department='Sales', employee_id='41-6841359', last_updated=datetime.date(2017, 1, 23), created=datetime.date(2016, 1, 27), vehicle_make='Ford', vehicle_model='Mustang', model_year=1997)
2 --> Combiter(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish', employer='Connelly Group', department='Research and Development', employee_id='98-7952860', last_updated=datetime.date(2017, 10, 4), created=datetime.date(2016, 9, 21), vehicle_make='GMC', vehicle_model='Yu

## Goal 3

Next, you want to identify any stale records, where stale simply means the record has not been updated since 3/1/2017 (e.g. last update date < 3/1/2017). Create an iterator that only contains current records (i.e. not stale) based on the `last_updated` field from the `status_update` file.

In [21]:
# A function to remove all the values below a particular date and return an iterator for the rest
def filiter(combi):
    date_ref = datetime.strptime('3/1/2017','%m/%d/%Y').date()
    for obj in combi:
        if obj.last_updated >= date_ref:
            yield obj
        else:
            continue

In [22]:
## Defining Individual Iterators
rows_perinfo = read_file('personal_info.csv', data_types_perinfo)
rows_employ = read_file('employment.csv', data_types_employ)
rows_updateStatus = read_file('update_status.csv', data_types_upstatus)
rows_vehicles = read_file('vehicles.csv', data_types_vehicles)

In [23]:
## Sorting the iterators
rows_perinfo_sort = sorting(rows_perinfo, 'ssn')
rows_employ_sort = sorting(rows_employ,'ssn')
rows_updateStatus_sort = sorting(rows_updateStatus,'ssn')
rows_vehicles_sort = sorting(rows_vehicles,'ssn')

In [24]:
## Combining the iterators
combi_iterators = combiter(rows_perinfo_sort,rows_employ_sort, rows_updateStatus_sort, rows_vehicles_sort)

In [25]:
## An iterator for the filtered dates
fil = filiter(combi_iterators)

In [26]:
## Printing the first 5 elements after filtering values less than a particular date
for index, rows in enumerate(islice(fil,5)):
    print(index,'-->',rows)

0 --> Combiter(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic', employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', last_updated=datetime.date(2017, 10, 7), created=datetime.date(2016, 1, 24), vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)
1 --> Combiter(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish', employer='Connelly Group', department='Research and Development', employee_id='98-7952860', last_updated=datetime.date(2017, 10, 4), created=datetime.date(2016, 9, 21), vehicle_make='GMC', vehicle_model='Yukon', model_year=2005)
2 --> Combiter(ssn='104-22-0928', first_name='Justinian', last_name='Kunzelmann', gender='Male', language='Dhivehi', employer='Upton LLC', department='Marketing', employee_id='56-9817552', last_updated=datetime.date(2017, 3, 28), created=datetime.date(2016, 4, 15), vehicle_make='Oldsmobile', vehicle_m

## Goal 4

Find the largest group of car makes for each gender.

Possibly more than one such group per gender exists (equal sizes).

In [27]:
## Function to count the cars used by Men and Women and return a dictionary with reverse sorted results
def counter(iterator):
    
    cars_male = dict()
    cars_female = dict()
    
    for item in iterator:
        if item.gender =='Male':
            try:
                cars_male[item.vehicle_model] +=1
            except:
                cars_male[item.vehicle_model] = 1
        else:
            try:
                cars_female[item.vehicle_model] +=1
            except:
                cars_female[item.vehicle_model] = 1
    
    return {k: v for k, v in sorted(cars_male.items(), key=lambda item: item[1], reverse=True)}, {k: v for k, v in sorted(cars_female.items(), key=lambda item: item[1], reverse = True)}

In [28]:
## Defining Individual Iterators
rows_perinfo = read_file('personal_info.csv', data_types_perinfo)
rows_employ = read_file('employment.csv', data_types_employ)
rows_updateStatus = read_file('update_status.csv', data_types_upstatus)
rows_vehicles = read_file('vehicles.csv', data_types_vehicles)

In [29]:
## Sorting the iterators
rows_perinfo_sort = sorting(rows_perinfo, 'ssn')
rows_employ_sort = sorting(rows_employ,'ssn')
rows_updateStatus_sort = sorting(rows_updateStatus,'ssn')
rows_vehicles_sort = sorting(rows_vehicles,'ssn')

In [30]:
## Combining the iterators
combi_iterators = combiter(rows_perinfo_sort,rows_employ_sort, rows_updateStatus_sort, rows_vehicles_sort)

In [31]:
## Counting the cars driven by Men and Women
Men, Women = counter(combi_iterators)

In [32]:
print ("Top 5 cars used by Men in the decreasing order: ", dict(list(Men.items())[0:5]) )

Top 5 cars used by Men in the decreasing order:  {'Savana 1500': 7, 'Mustang': 4, 'G': 4, 'Corvette': 4, 'Sonata': 4}


In [33]:
print ("Top 5 cars used by Women: ", dict(list(Women.items())[0: 5]))

Top 5 cars used by Women:  {'Mustang': 12, 'Esprit': 6, 'M5': 5, 'Corvette': 5, 'Taurus': 4}
